In [1]:
import cv2
import numpy as np
import mediapipe as mp
import pickle
import os
import time
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import pyttsx3 
import threading


tts_engine = pyttsx3.init()
tts_engine.setProperty('rate', 150)
tts_lock = threading.Lock()
last_spoken_gesture = None

def speak_text(text):
    global last_spoken_gesture
    with tts_lock:
        if text != last_spoken_gesture:
            last_spoken_gesture = text
            tts_engine.say(text)
            tts_engine.runAndWait()





class TFLiteModel:
    def __init__(self, model_path):
        self.interpreter = tf.lite.Interpreter(model_path=model_path)
        self.interpreter.allocate_tensors()
        self.input_details = self.interpreter.get_input_details()
        self.output_details = self.interpreter.get_output_details()
    
    def predict(self, input_data):
        input_data = np.array(input_data, dtype=self.input_details[0]['dtype'])
        if len(input_data.shape) == len(self.input_details[0]['shape']) - 1:
            input_data = np.expand_dims(input_data, axis=0)
        self.interpreter.set_tensor(self.input_details[0]['index'], input_data)
        self.interpreter.invoke()
        output_data = self.interpreter.get_tensor(self.output_details[0]['index'])
        return output_data
    


physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    try:
        for device in physical_devices:
            tf.config.experimental.set_memory_growth(device, True)
        print("GPU disponible para aceleración")
    except:
        print("Error al configurar GPU, usando CPU")   


mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=2,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)
mp_drawing = mp.solutions.drawing_utils


data_dir = "hand_gestures_data_4_1"
os.makedirs(data_dir, exist_ok=True)


model = None
scaler = StandardScaler()
label_encoder = LabelEncoder()
model_file = "hand_gesture_nn_model_4_1.h5"
scaler_file = "hand_gesture_scaler_4_1.pkl"
encoder_file = "hand_gesture_encoder_4_1.pkl"
gesture_data = "gesture_data_4_1.pkl"

data = []
labels = []
is_trained = False
is_collecting = False
current_gesture = ""
samples_collected = 0
max_samples = 100
last_sample_time = 0
sample_delay = 0.05
message = ""
message_until = 0
metrics = {
    'accuracy': 0,
    'val_accuracy': 0,
    'training_time': 0
}

def extract_hand_landmarks(frame):
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)
    landmarks_data = []
    hands_detected = False
    if results.multi_hand_landmarks:
        hands_detected = True
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            landmarks = []
            for landmark in hand_landmarks.landmark:
                landmarks.extend([landmark.x, landmark.y, landmark.z])
            landmarks_data.extend(landmarks)
    while len(landmarks_data) < 21 * 3 * 2:
        landmarks_data.append(0.0)
    landmarks_data = landmarks_data[:21 * 3 * 2]
    return landmarks_data, hands_detected

def set_message(message_text, duration=2):
    global message, message_until
    message = message_text
    message_until = time.time() + duration

def start_collection(gesture_name):
    global is_collecting, current_gesture, samples_collected
    is_collecting = True
    current_gesture = gesture_name
    samples_collected = 0
    set_message(f"Mantenga la seña frente a la cámara. Recolectando '{gesture_name}'...", 3)

def stop_collection():
    global is_collecting, current_gesture, samples_collected
    is_collecting = False
    current_gesture = ""
    samples_collected = 0
    set_message("Recolección finalizada", 2)


def save_data():
    global data, labels, gesture_data
    data_dict = {
        "features": data, 
        "labels": labels
        }
    with open(f"{data_dir}/{gesture_data}", "wb") as f:
        pickle.dump(data_dict, f)
    set_message(f"Datos guardados: {len(data)} muestras", 1)

def collect_sample(landmarks):
    global data, labels, samples_collected, last_sample_time, is_collecting
    if not is_collecting:
        return False
    current_time = time.time()
    if current_time - last_sample_time >= sample_delay:
        data.append(landmarks)
        labels.append(current_gesture)
        samples_collected += 1
        last_sample_time = current_time
        if samples_collected % 10 == 0:
            save_data()
        if samples_collected >= max_samples:
            stop_collection()
            return True
    return False


def load_data():
    global data, labels, gesture_data
    try:
        with open(f"{data_dir}/{gesture_data}", "rb") as f:
            data_dict = pickle.load(f)
            data = data_dict["features"]
            labels = data_dict["labels"]
        set_message(f"Datos cargados: {len(data)} muestras", 2)
        return True
    except:
        set_message("No se encontraron datos previos", 2)
        return False

def create_neural_network(input_shape, num_classes):
    model = Sequential([
        Dense(64, activation='relu', input_shape=(input_shape,), 
              kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        Dropout(0.3),
        Dense(32, activation='relu', kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        Dropout(0.2),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), 
                  loss='sparse_categorical_crossentropy', 
                  metrics=['accuracy'])
    return model

def train_model():
    global data, labels, scaler, label_encoder, is_trained, metrics
    if len(data) < 10:
        set_message("Se necesitan más datos para entrenar", 2)
        return 0
    X = np.array(data)
    y = np.array(labels)
    y_encoded = label_encoder.fit_transform(y)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
    )
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    num_classes = len(set(y_encoded))
    set_message(f"Entrenando modelo con {num_classes} clases...", 2)
    model = create_neural_network(X_train.shape[1], num_classes)
    callbacks = [
        EarlyStopping(
            monitor='val_loss', 
            patience=10, 
            restore_best_weights=True),
        ReduceLROnPlateau(
            monitor='val_loss', 
            factor=0.5, 
            patience=5, 
            min_lr=0.0001)
    ]
    start_time = time.time()
    history = model.fit(
        X_train, y_train, 
        epochs=50, 
        batch_size=32, 
        validation_split=0.2,
        callbacks=callbacks, 
        verbose=1)
    training_time = time.time() - start_time
    test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
    y_pred = np.argmax(model.predict(X_test), axis=-1)
    metrics.update({
        'accuracy': test_accuracy,
        'val_accuracy': max(history.history['val_accuracy']),
        'training_time': training_time,
        'confusion_matrix': confusion_matrix(y_test, y_pred),
        'report': classification_report(y_test, y_pred, target_names=label_encoder.classes_)
    })
    model.save(model_file)
    with open(scaler_file, 'wb') as f:
        pickle.dump(scaler, f)
    with open(encoder_file, 'wb') as f:
        pickle.dump(label_encoder, f)
    is_trained = True
    set_message(f"Modelo entrenado. Precisión: {test_accuracy:.2f}", 3)
    print(f"Precisión en datos de prueba: {test_accuracy:.4f}")
    print(f"Tiempo de entrenamiento: {training_time:.2f} segundos")
    print(metrics['report'])
    return test_accuracy

def load_saved_model():
    global scaler, label_encoder, is_trained
    try:
        model = load_model(model_file)
        with open(scaler_file, 'rb') as f:
            scaler = pickle.load(f)
        with open(encoder_file, 'rb') as f:
            label_encoder = pickle.load(f)
        is_trained = True
        set_message("Modelo de red neuronal cargado correctamente", 2)
        return model
    except Exception as e:
        return None

def predict_tflite(landmarks, tflite_model, scaler, label_encoder, threshold=0.9):
    X = np.array([landmarks])
    X_scaled = scaler.transform(X)
    prediction_probs = tflite_model.predict(X_scaled)[0]
    prediction_idx = np.argmax(prediction_probs)
    confidence = prediction_probs[prediction_idx]
    try:
        prediction_label = label_encoder.inverse_transform([prediction_idx])[0]
    except Exception as e:
        prediction_label = "Desconocido"
    if confidence >= threshold:
        return prediction_label, confidence
    return "Desconocido", confidence

def get_available_gestures():
    return list(set(labels))

def check_model_exists():
    return os.path.exists(model_file)

def convert_to_tflite(model_file, model_tflite):
    try:
        if not os.path.exists(model_file):
            raise FileNotFoundError(f"El archivo {model_file} no existe.")
        modelo = load_model(model_file)
        converter = tf.lite.TFLiteConverter.from_keras_model(modelo)
        tflite_model = converter.convert()
        with open(model_tflite, "wb") as f:
            f.write(tflite_model)
    except Exception as e:
        pass

model_path = "modelo_optimizadotl.tflite"
if not os.path.exists(model_path):
    pass
else:
    tflite_model = TFLiteModel(model_path)

def print_menu():
    print("\n=== MENU PRINCIPAL ===")
    print("1. Recolectar nueva seña")
    print("2. Entrenar modelo")
    print("3. Listar señas cargadas")
    print("4. Evaluar en tiempo real")
    print("5. Salir")

def list_gestures():
    if not labels:
        print("No hay señas guardadas.")
    else:
        unique_gestures = list(set(labels))
        print("\n--- Señas Guardadas ---")
        for i, gesture in enumerate(unique_gestures, 1):
            print(f"{i}. {gesture}")

def run_collection_mode():
    try:
        cap = cv2.VideoCapture(0)
    except Exception as e:
        return
    while is_collecting:
        ret, frame = cap.read()
        if not ret:
            time.sleep(0.1)
            continue
        landmarks, hands_detected = extract_hand_landmarks(frame)
        frame_h, frame_w, _ = frame.shape
        progress = int((samples_collected / max_samples) * frame_w)
        cv2.rectangle(frame, (0, 0), (progress, 20), (0, 255, 0), -1)
        cv2.putText(frame, f"Recolectando: {current_gesture} ({samples_collected}/{max_samples})", 
                    (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        if hands_detected:
            collect_sample(landmarks)
        else:
            cv2.putText(frame, "¡Muestra las manos!", (frame_w//2 - 100, frame_h//2), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        if not is_collecting:
            menu_active = True
            save_data()
        cv2.imshow("Recolectar Señas", frame)
        key = cv2.waitKey(1)
        if key == ord('m'):
            break
    cap.release()
    cv2.destroyAllWindows()
    save_data()

def run_evaluation_mode():
    global model_tflite
    if os.path.exists(model_tflite):
        tflite_model = TFLiteModel(model_tflite)
    else:
        return
    try:
        cap = cv2.VideoCapture(0)
    except Exception as e:
        return
    while True:
        ret, frame = cap.read()
        if not ret:
            time.sleep(0.1)
            continue
        landmarks, hands_detected = extract_hand_landmarks(frame)
        frame_h, frame_w, _ = frame.shape
        if hands_detected:
            prediction, confidence = predict_tflite(landmarks, tflite_model, scaler, label_encoder, threshold=0.9)
            color = (0, 255, 0) if confidence > 0.9 else (0, 165, 255)
            cv2.putText(frame, f"Seña: {prediction}", (10, 50), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
            cv2.putText(frame, f"Confianza: {confidence:.2%}", (10, 90), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
            confidence_value = np.max(confidence) if isinstance(confidence, np.ndarray) else confidence
            if confidence_value > 0.99 and prediction != "Desconocido":
                threading.Thread(target=speak_text, args=(prediction,), daemon=True).start()
        else:
            cv2.putText(frame, "Acerca las manos a la cámara", (frame_w//4, frame_h//2), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)
        cv2.putText(frame, "Presiona M para volver al menú", (10, frame_h - 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 1)
        cv2.imshow("Evaluación en Tiempo Real", frame)
        key = cv2.waitKey(1)
        if key == 27:  # ESC
            break

    cap.release()
    cv2.destroyAllWindows()
def main():
    global model, is_trained, data, labels

    is_trained = False
    model = None
    data = []
    labels = []

    load_data()
    
    if check_model_exists():
        model = load_saved_model()
        is_trained = True
    else:
        is_trained = False
    print_menu()

    while True:
        opcion = input("\nSelecciona una opción (Recolectar: 1, Entrenar: 2, Señas: 3, Evaluar: 4, Salir: 5): ").strip()
        
        if opcion == '1':
            gesture_name = input("Ingrese nombre de la seña (ej. 'Hola'): ")
            if gesture_name:
                start_collection(gesture_name)
                run_collection_mode()
                
        elif opcion == '2':
            if len(data) > 10:
                train_model()
                model = load_saved_model() if check_model_exists() else None
                is_trained = True
                print("Entrenamiento completado. Modelo entrenado.")
                convert_to_tflite(model_file, model_tflite)
                print("convertido a Tflite para evaluacion en tiemop real")
            else:
                print("¡Necesitas al menos 10 muestras para entrenar!")
                
        elif opcion == '3':
            list_gestures()
        elif opcion == '4':
            if is_trained:
                print("Modo de evaluación activado.")
                run_evaluation_mode()
            else:
                print("¡Entrena el modelo primero (Opción 2)!")
                
        elif opcion == '5':
            print("Saliendo del programa...")
            break
        else:
            print("Opción inválida, intenta nuevamente.")
        print_menu()

if __name__ == "__main__":
    main()


=== MENU PRINCIPAL ===
1. Recolectar nueva seña
2. Entrenar modelo
3. Listar señas cargadas
4. Evaluar en tiempo real
5. Salir
Epoch 1/50


c:\Users\juanp\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 66ms/step - accuracy: 0.4230 - loss: 1.6341 - val_accuracy: 0.5156 - val_loss: 1.1324 - learning_rate: 0.0010
Epoch 2/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.7435 - loss: 0.7396 - val_accuracy: 0.7656 - val_loss: 0.7812 - learning_rate: 0.0010
Epoch 3/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.8826 - loss: 0.5056 - val_accuracy: 0.8750 - val_loss: 0.6135 - learning_rate: 0.0010
Epoch 4/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.9273 - loss: 0.4108 - val_accuracy: 0.9531 - val_loss: 0.5090 - learning_rate: 0.0010
Epoch 5/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.9525 - loss: 0.3639 - val_accuracy: 0.9531 - val_loss: 0.4633 - learning_rate: 0.0010
Epoch 6/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.9303 - loss: 0.3628 - val_accuracy: 0.9531 - val_loss: 0.4403 - learning_rate: 0.0010
Epoch 7/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.9541 - loss: 0.3207 - val_accuracy: 0.9531 - val_loss:

Precisión en datos de prueba: 0.9625
Tiempo de entrenamiento: 10.59 segundos
              precision    recall  f1-score   support

        chau       1.00      1.00      1.00        20
  como estas       0.87      1.00      0.93        20
   construir       1.00      0.85      0.92        20
        hola       1.00      1.00      1.00        20

    accuracy                           0.96        80
   macro avg       0.97      0.96      0.96        80
weighted avg       0.97      0.96      0.96        80

Entrenamiento completado. Modelo entrenado.


NameError: name 'model_tflite' is not defined